## Pivot points & Trend Strategy

In [28]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import pandas_ta as ta

In [29]:
import os
from pathlib import Path
notebook_path = os.getcwd()
algo_dir = Path(notebook_path).parent.parent
csv_file = str(algo_dir) + '/vn-stock-data/VN30ps/VN30F1M_5minutes.csv'
is_file = os.path.isfile(csv_file)
if is_file:
    dataset = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
else:
    dataset = pd.read_csv("https://raw.githubusercontent.com/zuongthaotn/vn-stock-data/main/VN30ps/VN30F1M_5minutes.csv", index_col='Date', parse_dates=True)

In [30]:
_1_d_df = dataset.copy()
_1_d_df = _1_d_df.resample("D").agg({
    'Open': 'first',
    'Close': 'last',
    'High': 'max',
    'Low': 'min'
})
_1_d_df.dropna(inplace=True)
_1_d_df['High_s'] = _1_d_df['High'].shift(1)
_1_d_df['Low_s'] = _1_d_df['Low'].shift(1)
_1_d_df['Close_s'] = _1_d_df['Close'].shift(1)
_1_d_df['Pivot'] = _1_d_df.apply(
        lambda row: (row['High_s'] + row['Low_s'] + row['Close_s']) / 3, axis=1)
_1_d_df['R1'] = _1_d_df.apply(
        lambda row: 2 * row['Pivot'] - row['Low_s'], axis=1)
_1_d_df['R2'] = _1_d_df.apply(
        lambda row: row['Pivot'] + row['High_s'] - row['Low_s'], axis=1)
_1_d_df['R3'] = _1_d_df.apply(
        lambda row: row['Pivot'] * 2 + row['High_s'] - 2 * row['Low_s'], axis=1)
_1_d_df['S1'] = _1_d_df.apply(
        lambda row: 2 * row['Pivot'] - row['High_s'], axis=1)
_1_d_df['S2'] = _1_d_df.apply(
        lambda row: row['Pivot'] - (row['High_s'] - row['Low_s']), axis=1)
_1_d_df['S3'] = _1_d_df.apply(
        lambda row: row['Pivot'] * 2 - (2 * row['High_s'] - row['Low_s']), axis=1)
_1_d_df = _1_d_df[['S3', 'S2', 'S1', 'Pivot', 'R1', 'R2', 'R3']]

In [31]:
data = dataset.copy()
data["ma_line"] = data["Close"].rolling(20).mean()
data['above_ma'] = data.apply(lambda r: 1 if r['Close'] > r['ma_line'] else 0, axis=1)
data['below_ma'] = data.apply(lambda r: 1 if r['Close'] < r['ma_line'] else 0, axis=1)
data['total_above_ma'] = data['above_ma'].rolling(150).sum()
data['total_below_ma'] = data['below_ma'].rolling(150).sum()
data['trend'] = data.apply(lambda r: 'switch' if r['total_above_ma'] == r['total_below_ma'] else ('up' if r['total_above_ma'] > r['total_below_ma'] else 'down'), axis=1)
data['trend_shift'] = data['trend'].shift(1)
data['prev_Close'] = data['Close'].shift(1)
data.drop(columns=['ma_line', 'above_ma', 'below_ma'], inplace=True)
data = data.assign(time_d=pd.PeriodIndex(data.index, freq='1D').to_timestamp())
data = pd.merge(data, _1_d_df, left_on="time_d", right_index=True, how="left")
data.dropna(inplace=True)

In [32]:
data[(data.index > '2024-07-02 00:00:00') & (data.index < '2024-07-02 15:00:00')]

,Open,High,Low,Close,Volume,total_above_ma,total_below_ma,trend,trend_shift,prev_Close,time_d,S3,S2,S1,Pivot,R1,R2,R3
Date,,,,,,,,,,,,,,,,,,
2024-07-02 09:00:00,1284.7,1286.8,1283.5,1286.6,9220,63.0,87.0,down,down,1285.0,2024-07-02,1261.766667,1266.733333,1275.866667,1280.833333,1289.966667,1294.933333,1304.066667
2024-07-02 09:05:00,1286.7,1287.4,1286.6,1287.4,4444,63.0,87.0,down,down,1286.6,2024-07-02,1261.766667,1266.733333,1275.866667,1280.833333,1289.966667,1294.933333,1304.066667
2024-07-02 09:10:00,1287.3,1287.6,1287.1,1287.4,1916,64.0,86.0,down,down,1287.4,2024-07-02,1261.766667,1266.733333,1275.866667,1280.833333,1289.966667,1294.933333,1304.066667
2024-07-02 09:15:00,1287.3,1288.0,1286.2,1286.4,4382,65.0,85.0,down,down,1287.4,2024-07-02,1261.766667,1266.733333,1275.866667,1280.833333,1289.966667,1294.933333,1304.066667
2024-07-02 09:20:00,1286.6,1287.1,1286.2,1286.4,2175,66.0,84.0,down,down,1286.4,2024-07-02,1261.766667,1266.733333,1275.866667,1280.833333,1289.966667,1294.933333,1304.066667
2024-07-02 09:25:00,1286.4,1286.6,1284.9,1285.1,4732,67.0,83.0,down,down,1286.4,2024-07-02,1261.766667,1266.733333,1275.866667,1280.833333,1289.966667,1294.933333,1304.066667
2024-07-02 09:30:00,1285.2,1285.9,1285.1,1285.5,1859,68.0,82.0,down,down,1285.1,2024-07-02,1261.766667,1266.733333,1275.866667,1280.833333,1289.966667,1294.933333,1304.066667
2024-07-02 09:35:00,1285.4,1286.3,1285.0,1286.1,2391,69.0,81.0,down,down,1285.5,2024-07-02,1261.766667,1266.733333,1275.866667,1280.833333,1289.966667,1294.933333,1304.066667
2024-07-02 09:40:00,1286.2,1286.6,1285.7,1286.5,1808,70.0,80.0,down,down,1286.1,2024-07-02,1261.766667,1266.733333,1275.866667,1280.833333,1289.966667,1294.933333,1304.066667


## Strategy 02

In [58]:
def cal_signal_gen2(row):
    signal = ''
    if 100*row.name.hour + row.name.minute < 1425:
        if row['trend'] == 'up' and row['prev_Close'] < row['Pivot'] < row['Close']:
            signal = 'long'
        elif row['trend'] == 'down' and row['prev_Close'] > row['Pivot'] > row['Close']:
            signal = 'short'
        elif row['prev_Close'] < row['R1'] < row['Close'] or row['prev_Close'] < row['R2'] < row['Close'] or row['prev_Close'] <  row['R3'] < row['Close']:
            signal = 'long'
        elif row['prev_Close'] > row['S1'] > row['Close']  or row['prev_Close'] > row['S2'] > row['Close']  or row['prev_Close'] > row['S3'] > row['Close']:
            signal = 'short'
    elif 100*row.name.hour + row.name.minute == 1425:
        signal = 'close'
    return signal
def trade_simulation_gen2(prepared_data, takeprofit = True):
    _trades = pd.DataFrame(columns=("EntryTime", "EntryPrice", "Tp_step", "Sl_step", "ExitTime", "ExitPrice", "Type", "Profit"))
    has_open_deal = False
    type_open_deal = ''
    # Stoploss & Takeprofit are calculated auto after open deal(R/R = 1/3)
    sl_step = 0
    tp_step = 0
    for i, row in prepared_data.iterrows():
            if has_open_deal:
                if type_open_deal == 'short':
                    if row['signal'] == 'close':
                        # close open deal at 2:25PM (dataframe)
                        profit = entry_price - row['Close']
                        if profit < -sl_step:
                            profit = -sl_step
                        exit_price = row['Close']
                        exit_time = row.name
                        has_open_deal = False
                        type_open_deal = ''
                        _trades.loc[len(_trades)] = [entry_time, entry_price, tp_step, sl_step, exit_time, exit_price, "Short", profit]
                    # Stoploss
                    elif row['High'] >= entry_price + sl_step:
                        profit = -sl_step
                        exit_price = entry_price + sl_step
                        exit_time = row.name
                        _trades.loc[len(_trades)] = [entry_time, entry_price, tp_step, sl_step, exit_time, exit_price, "Short", profit]
                        has_open_deal = False
                        type_open_deal = ''
                    else:
                        if takeprofit:
                            # Takeprofit
                            if row['Low'] < entry_price - tp_step:
                                profit = tp_step
                                exit_price = entry_price - tp_step
                                exit_time = row.name
                                _trades.loc[len(_trades)] = [entry_time, entry_price, tp_step, sl_step, exit_time, exit_price, "Short", profit]
                                has_open_deal = False
                                type_open_deal = ''
                elif type_open_deal == 'long':
                    if row['signal'] == 'close':
                        # close open deal at 2:25PM (dataframe)
                        profit = row['Close'] - entry_price
                        if profit < -sl_step:
                            profit = -sl_step
                        exit_price = row['Close']
                        exit_time = row.name
                        has_open_deal = False
                        type_open_deal = ''
                        _trades.loc[len(_trades)] = [entry_time, entry_price, tp_step, sl_step, exit_time, exit_price, "Long", profit]
                    # Stoploss
                    if row['Low'] < entry_price - sl_step:
                        profit = -sl_step
                        exit_price = entry_price - sl_step
                        exit_time = row.name
                        _trades.loc[len(_trades)] = [entry_time, entry_price, tp_step, sl_step, exit_time, exit_price, "Long", profit]
                        has_open_deal = False
                        type_open_deal = ''
                    else:
                        if takeprofit:
                            # Takeprofit
                            if row['High'] > entry_price + tp_step:
                                profit = tp_step
                                exit_price = entry_price + tp_step
                                exit_time = row.name
                                _trades.loc[len(_trades)] = [entry_time, entry_price, tp_step, sl_step, exit_time, exit_price, "Long", profit]
                                has_open_deal = False
                                type_open_deal = ''
                        #
            if not has_open_deal:
                if row['signal'] == 'short':
                    # Open short deal
                    entry_price = row['Close']
                    entry_time = row.name
                    has_open_deal = True
                    type_open_deal = 'short'
                    if row['Pivot'] > entry_price > row['S1']:
                        tp_step = row['Pivot'] - (row['S1'] + 0.5)
                        sl_step = tp_step / 3
                    elif row['S1'] > entry_price > row['S2']:
                        tp_step = row['S1'] - (row['S2'] + 0.5)
                        sl_step = tp_step / 3
                    elif row['S2'] > entry_price > row['S3']:
                        tp_step = row['S2'] - (row['S3'] + 0.5)
                        sl_step = tp_step / 3
                    elif row['S3'] > entry_price:
                        tp_step = 3
                        sl_step = 1.5
                elif row['signal'] == 'long':
                    # Open short deal
                    entry_price = row['Close']
                    entry_time = row.name
                    has_open_deal = True
                    type_open_deal = 'long'
                    if row['Pivot'] < entry_price < row['R1']:
                        tp_step = row['R1'] - 0.5 - row['Pivot']
                        sl_step = tp_step / 3
                    elif row['R1'] < entry_price < row['R2']:
                        tp_step = row['R2'] - 0.5 - row['R1']
                        sl_step = tp_step / 3
                    elif row['R2'] < entry_price < row['R3']:
                        tp_step = row['R3'] - 0.5 - row['R2']
                        sl_step = tp_step / 3
                    elif row['R3'] > entry_price:
                        tp_step = 3
                        sl_step = 1.5
    return _trades

In [59]:
strategy02_data = data.copy()
strategy02_data['signal'] = strategy02_data.apply(lambda r: cal_signal_gen2(r), axis=1)
trade_result = trade_simulation_gen2(strategy02_data)
trade_result.Profit.sum()

1236.5222222222255

In [60]:
trade_result.Profit.count()

3597

In [61]:
# win rate
len(trade_result[trade_result.Profit > 0]) / (len(trade_result[trade_result.Profit < 0]) + len(trade_result[trade_result.Profit > 0]))

0.35638594534300055

In [62]:
trade_result.Profit.mean()

0.3437648657832153

In [63]:
trade_result

,EntryTime,EntryPrice,Tp_step,Sl_step,ExitTime,ExitPrice,Type,Profit
0,2018-08-16 09:10:00,941.0,4.733333,1.577778,2018-08-16 10:05:00,942.577778,Short,-1.577778
1,2018-08-16 10:20:00,940.8,4.733333,1.577778,2018-08-16 11:00:00,936.066667,Short,4.733333
2,2018-08-16 13:45:00,940.5,4.733333,1.577778,2018-08-16 13:50:00,942.077778,Short,-1.577778
3,2018-08-17 09:00:00,953.0,3.533333,1.177778,2018-08-17 09:45:00,951.822222,Long,-1.177778
4,2018-08-17 11:05:00,952.1,3.533333,1.177778,2018-08-17 13:30:00,950.922222,Long,-1.177778
...,...,...,...,...,...,...,...,...
3592,2024-06-28 13:10:00,1286.1,4.033333,1.344444,2024-06-28 13:40:00,1282.066667,Short,4.033333
3593,2024-06-28 14:15:00,1285.8,4.033333,1.344444,2024-06-28 14:20:00,1281.766667,Short,4.033333
3594,2024-06-28 14:20:00,1276.9,5.866667,1.955556,2024-06-28 14:25:00,1277.900000,Short,-1.000000
3595,2024-07-02 10:15:00,1290.8,4.466667,1.488889,2024-07-02 11:05:00,1295.266667,Long,4.466667


## Strategy 01

In [21]:
def trade_simulation(prepared_data, takeprofit = True):
    _trades = pd.DataFrame(columns=("EntryTime", "EntryPrice", "ExitTime", "ExitPrice", "Type", "Profit"))
    has_open_deal = False
    type_open_deal = ''
    # Stoploss at 20 pips
    sl_step = 3
    # Takeprofit at 60 pips(R/R = 1/3)
    tp_step = 9
    for i, row in prepared_data.iterrows():
            if has_open_deal:
                if type_open_deal == 'short':
                    if row['signal'] == 'close':
                        # close open deal at 2:25PM (dataframe)
                        profit = entry_price - row['Close']
                        if profit < -sl_step:
                            profit = -sl_step
                        exit_price = row['Close']
                        exit_time = row.name
                        has_open_deal = False
                        type_open_deal = ''
                        _trades.loc[len(_trades)] = [entry_time, entry_price, exit_time, exit_price, "Short", profit]
                    # Stoploss
                    elif row['High'] > entry_price + sl_step:
                        profit = -sl_step
                        exit_price = entry_price + sl_step
                        exit_time = row.name
                        _trades.loc[len(_trades)] = [entry_time, entry_price, exit_time, exit_price, "Short", profit]
                        has_open_deal = False
                        type_open_deal = ''
                    else:
                        if takeprofit:
                            # Takeprofit
                            if row['Low'] < entry_price - tp_step:
                                profit = tp_step
                                exit_price = entry_price - tp_step
                                exit_time = row.name
                                _trades.loc[len(_trades)] = [entry_time, entry_price, exit_time, exit_price, "Short", profit]
                                has_open_deal = False
                                type_open_deal = ''
                elif type_open_deal == 'long':
                    if row['signal'] == 'close':
                        # close open deal at 2:25PM (dataframe)
                        profit = row['Close'] - entry_price
                        if profit < -sl_step:
                            profit = -sl_step
                        exit_price = row['Close']
                        exit_time = row.name
                        has_open_deal = False
                        type_open_deal = ''
                        _trades.loc[len(_trades)] = [entry_time, entry_price, exit_time, exit_price, "Long", profit]
                    # Stoploss
                    if row['Low'] < entry_price - sl_step:
                        profit = -sl_step
                        exit_price = entry_price - sl_step
                        exit_time = row.name
                        _trades.loc[len(_trades)] = [entry_time, entry_price, exit_time, exit_price, "Long", profit]
                        has_open_deal = False
                        type_open_deal = ''
                    else:
                        if takeprofit:
                            # Takeprofit
                            if row['High'] > entry_price + tp_step:
                                profit = tp_step
                                exit_price = entry_price + tp_step
                                exit_time = row.name
                                _trades.loc[len(_trades)] = [entry_time, entry_price, exit_time, exit_price, "Long", profit]
                                has_open_deal = False
                                type_open_deal = ''
                        #
            if not has_open_deal:
                if row['signal'] == 'short':
                    # Open short deal
                    entry_price = row['Close']
                    entry_time = row.name
                    has_open_deal = True
                    type_open_deal = 'short'
                elif row['signal'] == 'long':
                    # Open short deal
                    entry_price = row['Close']
                    entry_time = row.name
                    has_open_deal = True
                    type_open_deal = 'long'
    return _trades

In [22]:
def cal_signal_01(row):
    signal = ''
    if 100*row.name.hour + row.name.minute == 900:
        if row['trend'] == 'up' and row['Close'] > row['Pivot']:
            signal = 'long'
        elif row['trend'] == 'down' and row['Close'] < row['Pivot']:
            signal = 'short'
    elif 100*row.name.hour + row.name.minute < 1425:
        if row['trend'] == 'up' and row['prev_Close'] < row['Pivot'] < row['Close']:
            signal = 'long'
        elif row['trend'] == 'down' and row['prev_Close'] > row['Pivot'] > row['Close']:
            signal = 'short'
        elif row['trend'] == 'switch' and row['trend_shift'] == 'up':
            signal = 'short'
        elif row['trend'] == 'switch' and row['trend_shift'] == 'down':
            signal = 'long'
    elif 100*row.name.hour + row.name.minute == 1425:
        signal = 'close'
    return signal

In [23]:
data['prev_Close'] = data['Close'].shift(1)
data['signal'] = data.apply(lambda r: cal_signal_01(r), axis=1)
trade_result = trade_simulation(data)
trade_result.Profit.sum()

982.0999999999983

In [24]:
trade_result.Profit.count()

2052

In [25]:
# win rate
len(trade_result[trade_result.Profit > 0]) / (len(trade_result[trade_result.Profit < 0]) + len(trade_result[trade_result.Profit > 0]))

0.36190009794319294

In [26]:
trade_result

,EntryTime,EntryPrice,ExitTime,ExitPrice,Type,Profit
0,2018-08-16 11:25:00,938.1,2018-08-16 13:35:00,941.1,Short,-3.0
1,2018-08-17 11:20:00,953.3,2018-08-17 13:35:00,950.3,Long,-3.0
2,2018-08-20 09:00:00,945.8,2018-08-20 11:15:00,948.8,Short,-3.0
3,2018-08-20 13:55:00,946.5,2018-08-20 14:25:00,944.7,Short,1.8
4,2018-08-21 09:00:00,944.5,2018-08-21 10:30:00,947.5,Short,-3.0
...,...,...,...,...,...,...
2047,2024-06-27 13:40:00,1285.2,2024-06-27 14:00:00,1288.2,Short,-3.0
2048,2024-06-28 09:15:00,1292.0,2024-06-28 09:40:00,1289.0,Long,-3.0
2049,2024-06-28 10:15:00,1287.9,2024-06-28 14:20:00,1278.9,Short,9.0
2050,2024-07-01 09:00:00,1277.5,2024-07-01 14:00:00,1280.5,Short,-3.0


In [27]:
trade_result.Profit.mean()

0.4786062378167633